In [0]:
from pyspark.sql.functions import (
    sequence, explode, to_date, date_format,
    day, dayofweek, weekofyear, month,
    quarter, year, when, col, expr
)

# -------------------------------------------------
# Date range
# -------------------------------------------------
start_date = "2015-01-01"
end_date = "2035-12-31"

# -------------------------------------------------
# Generate date sequence
# -------------------------------------------------
dates_df = (
    spark.createDataFrame([(start_date, end_date)], ["start", "end"])
    .select(
        explode(
            sequence(
                to_date(col("start")),
                to_date(col("end")),
                expr("interval 1 day")
            )
        ).alias("FullDate")
    )
)

# -------------------------------------------------
# Build DimDate attributes
# -------------------------------------------------
dim_date_df = (
    dates_df
    .withColumn("DateKey", date_format(col("FullDate"), "yyyyMMdd").cast("int"))
    .withColumn("Day", day(col("FullDate")))
    .withColumn("DayName", date_format(col("FullDate"), "EEEE"))
    .withColumn("DayOfWeek", dayofweek(col("FullDate")))
    .withColumn("WeekOfYear", weekofyear(col("FullDate")))
    .withColumn("Month", month(col("FullDate")))
    .withColumn("MonthName", date_format(col("FullDate"), "MMMM"))
    .withColumn("Quarter", quarter(col("FullDate")))
    .withColumn("Year", year(col("FullDate")))
    .withColumn(
        "IsWeekend",
        when(dayofweek(col("FullDate")).isin([1, 7]), "Y").otherwise("N")
    )
    .select(
        "DateKey",
        "FullDate",
        "Day",
        "DayName",
        "DayOfWeek",
        "WeekOfYear",
        "Month",
        "MonthName",
        "Quarter",
        "Year",
        "IsWeekend"
    )
)

# -------------------------------------------------
# Write to Gold layer (Unity Catalog)
# -------------------------------------------------
(
    dim_date_df
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.gold.dim_date")
)


In [0]:
spark.table("workspace.gold.dim_date").count()


In [0]:
spark.table("workspace.gold.dim_date").orderBy("FullDate").show(10, truncate=False)
